In [1]:
%matplotlib inline
import mxnet as mx
import numpy as np
import logging
import matplotlib.pyplot as plt

In [2]:
dev = mx.gpu()
batch_size = 100
data_shape = (1,28,28)
batch_shape = (100,1,28,28)
train_iter = mx.io.MNISTIter(
        image       = "../mxnet/mnist/train-images-idx3-ubyte",
        label       = "../mxnet/mnist/train-labels-idx1-ubyte",
        input_shape = data_shape,
        batch_size  = batch_size,
        shuffle     = True,
        flat        = False,
        ctx = dev)

val_iter = mx.io.MNISTIter(
        image       = "../mxnet/mnist/t10k-images-idx3-ubyte",
        label       = "../mxnet/mnist/t10k-labels-idx1-ubyte",
        input_shape = data_shape,
        batch_size  = batch_size,
        flat        = False,
        ctx = dev)

# Define Networks

In [3]:
def conv2d_relu(data, name, **kwargs):
    net = mx.sym.Convolution(data, name = "%s_conv"%name, **kwargs)
    net = mx.sym.Activation(net, act_type='relu', name = "%s_act"%name)
    return net
def conv2d_act(data, name, act, **kwargs):
    net = mx.sym.Convolution(data, name = "%s_conv"%name, **kwargs)
    net = mx.sym.Activation(net, act_type=act, name = "%s_act"%name)
    return net

In [102]:
def Classifier(data = None, num_hidden=256, ngf=20):
    data = mx.sym.Variable('data') if data is None else data
    net = conv2d_relu(data, kernel = (4,4), stride = (2,2), pad = (1,1), num_filter=ngf, name = 'EC_conv1')
    net = conv2d_relu(net, kernel = (4,4), stride = (2,2), pad = (2,2), num_filter=50, name = 'EC_conv2')
    #net = conv2d_relu(net, kernel = (4,4), stride = (2,2), pad = (1,1), num_filter=ngf*4, name = 'EC_conv3')
    net = mx.sym.Flatten(net)
    net = mx.sym.Activation(net, name = 'EC_act1', act_type = 'relu')    
    net = mx.sym.FullyConnected(data, num_hidden = num_hidden, name = "C_fc1")
    net = mx.sym.Activation(net, name = 'EC_act1', act_type = 'relu')    
    net = mx.sym.FullyConnected(data, num_hidden = 64, name = "C_fc1")
    net = mx.sym.Activation(net, name = 'C_act1', act_type = 'relu')
    net = mx.sym.FullyConnected(net, num_hidden = 10, name = 'C_fc_2')
    #net = mx.sym.SoftmaxOutput(net, name = 'C_softmax')
    return net

## Aux Functions

In [5]:
def CalAcc(prob, label):
    return np.sum(np.argmax(prob, axis=1)==label)*1.0/batch_size
def EDLoss(decoder, data):
    res = 0.0
    
    temp = decoder - data
    for j in range(batch_size):
        res += mx.nd.norm(temp[j])
    return res/batch_size

def CLoss(prob, label):
    res = 0.0
    for j in range(batch_size):
        res -= np.log(prob[j][int(label[j])])
    return res/batch_size

def SGD(weight, grad, lr = 0.05, wd = 0.0001):
    weight[:] -= lr*(grad/batch_size + wd*weight) 
    
def softmax(theta):
    tmp = theta - np.max(theta, axis=1, keepdims = True)
    exp = np.exp(tmp)
    norm = np.sum(exp, axis=1, keepdims = True)
    return exp/norm
    
def logLossGrad(alpha,label):
    res = np.copy(alpha)
    for j in range(alpha.shape[0]):
        res[j][int(label[j])] -= 1
    return res

def Validate_loss(val_iter):
    val_loss = 0.0
    val_iter.reset()
    nbatch = 0
    for dbatch in val_iter:
        data = dbatch.data[0]
        E_arg_map['data'][:] = data
        modE.forward(is_train=False)
        coder = modE.outputs[0]
        
        D_arg_map['data'][:] = coder
        modD.forward(is_train = False)
        decoder = modD.outputs[0]
        
        data_gpu = mx.nd.zeros(shape = data.shape, ctx = dev)
        data.copyto(data_gpu)
        val_loss += EDLoss(decoder, data_gpu)
        nbatch +=1
    return val_loss/nbatch

In [78]:
def acc_normal(model, val_iter, arg_map):
    val_iter.reset()
    val_acc = 0.0
    num_samp = 0
    for dbatch in val_iter:
        data = dbatch.data[0]
        label = dbatch.label[0]
        batch_size = label.asnumpy().shape[0]
        #print data.shape
        #print arg_map['data'].shape
        arg_map["data"][:] = data    

        model.forward(is_train=False)
        theta = model.outputs[0].asnumpy()
        alpha = softmax(theta)
        val_acc += CalAcc(alpha, label.asnumpy()) 
        num_samp += 1
    return(val_acc / num_samp)


# Generate Fixed Perturbed Data

In [43]:
data = mx.symbol.Variable('data')
# first conv
conv1 = mx.symbol.Convolution(data=data, kernel=(5,5), num_filter=20)
tanh1 = mx.symbol.Activation(data=conv1, act_type="tanh")
pool1 = mx.symbol.Pooling(data=tanh1, pool_type="max",
                              kernel=(2,2), stride=(2,2))
# second conv
conv2 = mx.symbol.Convolution(data=pool1, kernel=(5,5), num_filter=50)
tanh2 = mx.symbol.Activation(data=conv2, act_type="tanh")
pool2 = mx.symbol.Pooling(data=tanh2, pool_type="max",
                              kernel=(2,2), stride=(2,2))
# first fullc
flatten = mx.symbol.Flatten(data=pool2)
fc1 = mx.symbol.FullyConnected(data=flatten, num_hidden=100)
tanh3 = mx.symbol.Activation(data=fc1, act_type="tanh")
# second fullc
fc2 = mx.symbol.FullyConnected(data=tanh3, num_hidden=10)

In [44]:
data_shape = (batch_size, 1, 28, 28)
arg_names = fc2.list_arguments() # 'data' 
arg_shapes, output_shapes, aux_shapes = fc2.infer_shape(data=data_shape)

arg_arrays = [mx.nd.zeros(shape, ctx=dev) for shape in arg_shapes]
grad_arrays = [mx.nd.zeros(shape, ctx=dev) for shape in arg_shapes]
reqs = ["write" for name in arg_names]

model = fc2.bind(ctx=dev, args=arg_arrays, args_grad = grad_arrays, grad_req=reqs)
arg_map = dict(zip(arg_names, arg_arrays))
grad_map = dict(zip(arg_names, grad_arrays))
data_grad = grad_map["data"]
out_grad = mx.nd.zeros(model.outputs[0].shape, ctx=dev)

In [45]:
for name in arg_names:
    if "weight" in name:
        arr = arg_map[name]
        arr[:] = mx.rnd.uniform(-0.07, 0.07, arr.shape)

In [46]:
num_round = 20
lr = 0.2

Training_normal = np.zeros(shape = (num_round))
Validation_normal = np.zeros(shape = (num_round))

for i in range(num_round):
    train_iter.reset()
    train_acc = 0.0
    loss_total = 0.0
    num_batch = 0
    
    for dbatch in train_iter:
        data = dbatch.data[0]
        label = dbatch.label[0]
        
        arg_map['data'][:] = data
        model.forward(is_train=True)
        theta = model.outputs[0].asnumpy()
        alpha = softmax(theta)
        
        loss_total +=  CLoss(alpha, label.asnumpy())
        train_acc += CalAcc(alpha, label.asnumpy())
        logGrad = logLossGrad(alpha, label.asnumpy())
        
        out_grad[:] = logGrad
        model.backward([out_grad])
                
        for name in arg_names:
            if name!='data':
                SGD(arg_map[name], grad_map[name], lr)      

        num_batch +=1
        if num_batch % 300==299:
            print "Training Loss: %.4f\t Training Accuracy: %.4f" %(loss_total/num_batch,train_acc/num_batch)
    Training_normal[i] = train_acc/num_batch
    print acc_normal(model, val_iter, arg_map)

Training Loss: 0.4254	 Training Accuracy: 0.8816
Training Loss: 0.2664	 Training Accuracy: 0.9250
0.9753
Training Loss: 0.0782	 Training Accuracy: 0.9762
Training Loss: 0.0680	 Training Accuracy: 0.9798
0.9846
Training Loss: 0.0509	 Training Accuracy: 0.9853
Training Loss: 0.0460	 Training Accuracy: 0.9868
0.9868
Training Loss: 0.0378	 Training Accuracy: 0.9895
Training Loss: 0.0348	 Training Accuracy: 0.9904
0.9881
Training Loss: 0.0295	 Training Accuracy: 0.9919
Training Loss: 0.0274	 Training Accuracy: 0.9927
0.9898
Training Loss: 0.0235	 Training Accuracy: 0.9938
Training Loss: 0.0221	 Training Accuracy: 0.9944
0.9901
Training Loss: 0.0189	 Training Accuracy: 0.9951
Training Loss: 0.0179	 Training Accuracy: 0.9956
0.9907
Training Loss: 0.0154	 Training Accuracy: 0.9963
Training Loss: 0.0148	 Training Accuracy: 0.9967
0.9911
Training Loss: 0.0128	 Training Accuracy: 0.9972
Training Loss: 0.0123	 Training Accuracy: 0.9975
0.991
Training Loss: 0.0107	 Training Accuracy: 0.9979
Trainin

In [93]:
val_iter.reset()
val_acc = 0.0
val_acc_pb = 0.0
coe_pb = 2
num_batch = 0

perb_data = []
perb_lab = []

for dbatch in val_iter:
    data = dbatch.data[0]
    label = dbatch.label[0]
    arg_map["data"][:] = data    
    
    model.forward(is_train=True)
    theta = model.outputs[0].asnumpy()
    alpha = softmax(theta)
    
    val_acc += CalAcc(alpha, label.asnumpy())
    logGrad = logLossGrad(alpha, label.asnumpy())
        
    out_grad[:] = logGrad
    model.backward([out_grad])
    noise = data_grad.asnumpy()
    for j in range(batch_size):
        #noise[j] = np.sign(noise[j])
        if np.linalg.norm(noise[j].flatten(),2) != 0:
            noise[j] = noise[j]/np.linalg.norm(noise[j].flatten(),2)
    pdata = data.asnumpy() + coe_pb * noise
    arg_map["data"][:] = pdata
    model.forward(is_train=True)
    raw_output = model.outputs[0].asnumpy()
    pred = softmax(raw_output)
    val_acc_pb += CalAcc(pred, label.asnumpy()) 
    num_batch += 1
    pdata = np.minimum(np.maximum(pdata,0),1)
    perb_data.append(pdata)
    perb_lab.append(label.asnumpy())
    #print np.max(np.absolute(noise[3][0]))
print("Val Batch Accuracy: ", val_acc / num_batch)
print("Val Batch Accuracy after pertubation: ", val_acc_pb / num_batch)


('Val Batch Accuracy: ', 0.99129999999999951)
('Val Batch Accuracy after pertubation: ', 0.25420000000000009)


In [94]:
pdata = np.concatenate(perb_data, axis = 0)
plabel = np.concatenate(perb_lab, axis = 0)
perb_iter = mx.io.NDArrayIter(
    data = pdata,
    label = plabel,
    batch_size = 100,
    shuffle = False    
)

In [95]:
perb_iter.reset()
num_samp = 0
val_acc = 0.0
for dbatch in perb_iter:
    data = dbatch.data[0]
    label = dbatch.label[0]
    arg_map["data"][:] = data    
    
    model.forward(is_train=True)
    theta = model.outputs[0].asnumpy()
    alpha = softmax(theta)
    val_acc += CalAcc(alpha, label.asnumpy()) 
    num_samp += 1
print("Val Batch Accuracy after pertubation: ", val_acc / num_samp)

('Val Batch Accuracy after pertubation: ', 0.64260000000000006)


## Initialization

In [103]:
C_net = Classifier()
C_arg_names = C_net.list_arguments()
C_arg_shapes, C_output_shapes, C_aux_shapes = C_net.infer_shape(data = batch_shape)
C_arg_arrays = [mx.nd.zeros(shape, ctx=dev) for shape in C_arg_shapes]
C_grad_arrays = [mx.nd.zeros(shape, ctx=dev) for shape in C_arg_shapes]
C_aux_states =  [mx.nd.zeros(shape, ctx=dev) for shape in C_aux_shapes]
C_reqs = ["write" for name in C_arg_names]

modC = C_net.bind(ctx=dev, args=C_arg_arrays, args_grad = C_grad_arrays, grad_req=C_reqs,  aux_states=C_aux_states)
C_arg_map = dict(zip(C_arg_names, C_arg_arrays))
C_grad_map = dict(zip(C_arg_names, C_grad_arrays))
C_data_grad = C_grad_map["data"]
C_out_grad = mx.nd.zeros(modC.outputs[0].shape, ctx=dev)

## Normal Training

In [104]:
mx.rnd.seed(17214)
for name in C_arg_names:
    if "weight" in name:
        arr = C_arg_map[name]
        shape = arr.shape
        fan_in, fan_out = np.prod(shape[1:]), shape[0]
        factor = fan_in
        scale = np.sqrt(2.34 / factor)
        arr[:] = mx.rnd.uniform(-scale, scale, arr.shape)
    else:
        arr = C_arg_map[name]
        arr[:] = 0.

In [ ]:
num_epoch = 90
lr = 0.1

Training_normal = np.zeros(shape = (num_epoch))
Validation_normal = np.zeros(shape = (num_epoch))
Adv_normal = np.zeros(shape = (num_epoch))

for i in range(num_epoch):
    if i%30==29: 
        lr = lr/2.0
    train_iter.reset()
    train_acc = 0.0
    loss_total = 0.0
    num_batch = 0
    
    for dbatch in train_iter:
        data = dbatch.data[0]
        label = dbatch.label[0]
        
        C_arg_map['data'][:] = data
        modC.forward(is_train=True)
        theta = modC.outputs[0].asnumpy()
        alpha = softmax(theta)
        
        loss_total +=  CLoss(alpha, label.asnumpy())
        train_acc += CalAcc(alpha, label.asnumpy())
        logGrad = logLossGrad(alpha, label.asnumpy())
        
        C_out_grad[:] = logGrad
        modC.backward([C_out_grad])
                
        for name in C_arg_names:
            if name!='data':
                SGD(C_arg_map[name], C_grad_map[name], lr)
                
        num_batch +=1
        if num_batch % 300==299:
            print "Training Loss: %.4f\t Training Accuracy: %.4f" %(loss_total/num_batch,train_acc/num_batch)
    Training_normal[i] = train_acc/num_batch
    Validation_normal[i] = acc_normal(modC, val_iter, C_arg_map)
    Adv_normal[i] = acc_normal(modC, perb_iter, C_arg_map)
    print "epoch: %d Validation Accuracy: %.4f\t Adverserial Accuracy: %.4f" \
        %(i, Validation_normal[i], Adv_normal[i])

## Discretize Training

In [99]:
def discretize(data, p):
    tmp = np.zeros(shape = data.shape)
    tmp[:] = np.floor(data*p)
    tmp = tmp/p
    return tmp

def acc_dis(model, val_iter, arg_map,p):
    val_iter.reset()
    val_acc = 0.0
    num_samp = 0
    for dbatch in val_iter:
        data = dbatch.data[0]
        label = dbatch.label[0]
        batch_size = label.asnumpy().shape[0]
        
        tmp = discretize(data.asnumpy(),p)
        arg_map["data"][:] = tmp    

        model.forward(is_train=False)
        theta = model.outputs[0].asnumpy()
        alpha = softmax(theta)
        val_acc += CalAcc(alpha, label.asnumpy()) 
        num_samp += 1
    return(val_acc / num_samp)

In [100]:
mx.rnd.seed(214)
for name in C_arg_names:
    if "weight" in name:
        arr = C_arg_map[name]
        shape = arr.shape
        fan_in, fan_out = np.prod(shape[1:]), shape[0]
        factor = fan_in
        scale = np.sqrt(2.34 / factor)
        arr[:] = mx.rnd.uniform(-scale, scale, arr.shape)
    else:
        arr = C_arg_map[name]
        arr[:] = 0.

In [101]:
num_epoch = 120
lr = 0.1
p = 3.0

Training_dis = np.zeros(shape = (num_epoch))
Validation_dis = np.zeros(shape = (num_epoch))
Adv_dis = np.zeros(shape = (num_epoch))

for i in range(num_epoch):
    if i%30==29: 
        lr = lr/2.0
    train_iter.reset()
    train_acc = 0.0
    loss_total = 0.0
    num_batch = 0
    
    for dbatch in train_iter:
        data = dbatch.data[0]
        label = dbatch.label[0]
        
        tmp = discretize(data.asnumpy(),p)
        
        C_arg_map['data'][:] = tmp
        modC.forward(is_train=True)
        theta = modC.outputs[0].asnumpy()
        alpha = softmax(theta)
        
        loss_total +=  CLoss(alpha, label.asnumpy())
        train_acc += CalAcc(alpha, label.asnumpy())
        logGrad = logLossGrad(alpha, label.asnumpy())
        
        C_out_grad[:] = logGrad
        modC.backward([C_out_grad])
                
        for name in C_arg_names:
            if name!='data':
                SGD(C_arg_map[name], C_grad_map[name], lr)
                
        num_batch +=1
        if num_batch % 300==299:
            print "Training Loss: %.4f\t Training Accuracy: %.4f" %(loss_total/num_batch,train_acc/num_batch)
    Training_dis[i] = train_acc/num_batch
    Validation_dis[i] = acc_dis(modC, val_iter, C_arg_map,p)
    Adv_dis[i] = acc_dis(modC, perb_iter, C_arg_map,p)
    print "epoch: %d Validation Accuracy: %.4f\t Adverserial Accuracy: %.4f" \
        %(i, Validation_dis[i], Adv_dis[i])

Training Loss: 0.7884	 Training Accuracy: 0.8111
Training Loss: 0.5817	 Training Accuracy: 0.8525
epoch: 0 Validation Accuracy: 0.9089	 Adverserial Accuracy: 0.8504
Training Loss: 0.3116	 Training Accuracy: 0.9123
Training Loss: 0.3028	 Training Accuracy: 0.9143
epoch: 1 Validation Accuracy: 0.9262	 Adverserial Accuracy: 0.8637
Training Loss: 0.2583	 Training Accuracy: 0.9275
Training Loss: 0.2536	 Training Accuracy: 0.9286
epoch: 2 Validation Accuracy: 0.9340	 Adverserial Accuracy: 0.8721
Training Loss: 0.2229	 Training Accuracy: 0.9367
Training Loss: 0.2196	 Training Accuracy: 0.9381
epoch: 3 Validation Accuracy: 0.9401	 Adverserial Accuracy: 0.8788
Training Loss: 0.1970	 Training Accuracy: 0.9442
Training Loss: 0.1942	 Training Accuracy: 0.9452
epoch: 4 Validation Accuracy: 0.9469	 Adverserial Accuracy: 0.8830
Training Loss: 0.1762	 Training Accuracy: 0.9503
Training Loss: 0.1738	 Training Accuracy: 0.9511
epoch: 5 Validation Accuracy: 0.9513	 Adverserial Accuracy: 0.8871
Training L

In [84]:
val_iter.reset()
perb_iter.reset()
data1 = val_iter.next().data[0]
data2 = perb_iter.next().data[0]
sample1 = data1[3][0].asnumpy()
sample2 = data2[3][0].asnumpy()